In [1]:
# my_label_path="/home/aikedaer/mydata/data/lingodt/val.csv"
# my_image_path="/home/aikedaer/mydata/data/lingodt"

# my_label_path="/home/aikedaer/mydata/data/comp/question1.csv"
# my_image_path="/home/aikedaer/mydata/data/comp"
# qton = "question1"
type_method = "sga_albef"
my_label_path="/home/aikedaer/mydata/data/comp/question2.csv"
my_image_path=f"/home/aikedaer/mydata/data/comp/attacked/phase2/{type_method}"
qton = "question2" 

In [2]:
# !pip install transformers==v4.39.2
# !pip install -q pillow acceleratea einops
# export CUDA_VISIBLE_DEVICES=1

In [3]:
!pip show transformers

Name: transformers
Version: 4.39.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /mnt/data1/aikedaer/anaconda3/envs/imp/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: transformers-stream-generator


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image
# torch.set_default_device("cuda")
default_device = torch.device("cuda:0")
torch.cuda.set_device(default_device)
torch.__version__


/mnt/data1/aikedaer/anaconda3/envs/imp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'2.3.0+cu121'

In [5]:
# !git clone https://huggingface.co/MILVLG/imp-v1-3b

In [6]:
# !huggingface-cli download --resume-download MILVLG/imp-v1-3b

In [7]:
#Create model
# import os
# os.environ['HF_ENDPOINT'] = 'hf-mirror.com'

model = AutoModelForCausalLM.from_pretrained(
    "MILVLG/imp-v1-3b", 
    torch_dtype=torch.float16, 
    device_map="auto",
    trust_remote_code=True,
    # force_download=True,
    resume_download=True)
tokenizer = AutoTokenizer.from_pretrained("MILVLG/imp-v1-3b", trust_remote_code=True)


/mnt/data1/aikedaer/anaconda3/envs/imp/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# !pip install pyarrow fastparquet -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
# !pip install pandas -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
import pandas as pd
import os
from tqdm import tqdm 

def get_answer(img_path, question):
    text = f"A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. \
            USER: <image>\n{question} ASSISTANT:"
    file_pth = os.path.join(my_image_path,img_path)
    image0 = Image.open(file_pth) 
    input_ids = tokenizer(text, return_tensors='pt').input_ids.to(default_device)
    image_tensor0 = model.image_preprocess(image0).to(default_device)

    #Generate the answer
    output_ids = model.generate(
        input_ids,
        max_new_tokens=50,
        images=image_tensor0,
        use_cache=True)[0]
    asw = tokenizer.decode(output_ids[input_ids.shape[1]:], skip_special_tokens=True).strip()

    return asw
def str2list(input_string):
    # Remove the brackets and split by newline character
    list_elements = input_string.strip("[]").split('\n')

    # Remove any extra spaces and quotes
    cleaned_list = [element.strip(" '") for element in list_elements]
    return cleaned_list

val = pd.read_csv(my_label_path)
val["images"] = val["images"].map(str2list)
predictions = pd.DataFrame(columns=["question_id", "segment_id", "answer"])

for index, row in tqdm(val.iterrows(), total=val.shape[0]):
    question_id = row['question_id']
    segment_id = row['segment_id']
    images = row['images']
    question = row['question']
    answer = row['answer']
    # only use the first frame
    asw = get_answer(images[0], question)
    # Create a DataFrame for the current row
    new_row = pd.DataFrame({
        "question_id": [question_id],
        "segment_id": [segment_id],
        "question": [question],
        "answer": [asw]
    })

    # Append the new row to the predictions DataFrame using pd.concat
    predictions = pd.concat([predictions, new_row], ignore_index=True)
    # break

predictions



100%|██████████| 922/922 [1:53:34<00:00,  7.39s/it]


,question_id,segment_id,answer,question
0,xxxxxxxxxxx_0,xxxxxxxxxxxxx_0,"In the image, there are no cars, traffic light...","how many cars, traffic lights, road signals, p..."
1,xxxxxxxxxxx_1,xxxxxxxxxxxxx_1,"The cars in the image are black, the traffic l...","what colors are the cars, traffic lights, road..."
2,xxxxxxxxxxx_2,xxxxxxxxxxxxx_2,"There is one car, one traffic light, one road ...","how many cars, traffic lights, road signals, p..."
3,xxxxxxxxxxx_3,xxxxxxxxxxxxx_3,"The cars in the image are red, the traffic lig...","what colors are the cars, traffic lights, road..."
4,xxxxxxxxxxx_4,xxxxxxxxxxxxx_4,"In the image, there is one car, two traffic li...","how many cars, traffic lights, road signals, p..."
...,...,...,...,...
917,xxxxxxxxxxx_917,xxxxxxxxxxxxx_917,"The cars in the image are black, the traffic l...","what colors are the cars, traffic lights, road..."
918,xxxxxxxxxxx_918,xxxxxxxxxxxxx_918,"The image does not show any cars, traffic ligh...","how many cars, traffic lights, road signals, p..."
919,xxxxxxxxxxx_919,xxxxxxxxxxxxx_919,"The cars in the image are black, the traffic l...","what colors are the cars, traffic lights, road..."
920,xxxxxxxxxxx_920,xxxxxxxxxxxxx_920,"In the image, there are no cars, traffic light...","how many cars, traffic lights, road signals, p..."


In [ ]:
my_new = f"/home/aikedaer/mydata/data/comp/predict/{type_method}/{qton}_imp_predict.csv"
predictions.to_csv(my_new)